#### Mapping a Class against Multiple Tables

_Mappers_ can be constructed against arbitrary relational units (called _selectables_) in addition to plain tables. For example, the `join()` function _creates a selectable unit comprised of multiple tables_, complete with its own __composite primary key__, which can be mapped in the same way as a `Table`.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, MetaData, String, Table, join
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import column_property

In [2]:
metadata_obj = MetaData()
Base = declarative_base()

In [3]:
user_table = Table(
    "user",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [4]:
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("user.id")),
    Column("email_address", String),
)

In [5]:
user_address_join = join(user_table, address_table)

In [6]:
class AddressUser(Base):
    __table__ = user_address_join
    
    id = column_property(user_table.c.id, address_table.c.id)
    address_id = address_table.c.id

In the example above, the _join_ expresses columns for both the _user_ and the _address_ table. The `user.id` and `address.user_id` columns are __equated by foreign key__, so _in the mapping_ they are __defined as one attribute__, `AddressUser.id`, using `column_property()` to __indicate a specialized column mapping__. Based on this part of the _configuration_, the _mapping_ will __copy new primary key values__ *from user.id into the address.user_id* column when a flush occurs.

Additionally, the `address.id` column is __mapped explicitly__ to an attribute named `address_id`. This is to __disambiguate__ the mapping of the `address.id` column from the _same-named_ `AddressUser.id` attribute, which here has been assigned to __refer to the user table combined with the address.user_id foreign key__.

The _natural primary key_ of the above mapping is the _composite_ of (`user.id`, `address.id`), as these are the _primary key_ columns of the _user_ and _address_ table __combined together__. The identity of an `AddressUser` object will be in terms of these two values, and is represented from an `AddressUser` object as (`AddressUser.id`, `AddressUser.address_id`).

When referring to the `AddressUser.id` column, _most SQL expressions_ will make use of __only the first column__ in the list of columns mapped, as the _two columns_ are __synonymous__. However, for the special use case such as a `GROUP BY` expression where __both columns must be referenced__ at the _same time_ while making use of the _proper context_, that is, accommodating for _aliases and similar_, the accessor `Comparator.expressions` may be used:

```
q = session.query(AddressUser).group_by(*AddressUser.id.expressions)
```

> ##### Note
>
> A _mapping against multiple tables_ as illustrated above __supports persistence__, that is, _INSERT, UPDATE and DELETE_ of rows within the targeted tables. However, it __does not support__ an operation that would _UPDATE one table_ and _perform INSERT or DELETE on others_ at the same time for one record. That is, if a record _PtoQ_ is mapped to tables `"p"` and `"q"`, where it has a row based on a _LEFT OUTER JOIN_ of `"p" and "q"`, if an _UPDATE_ proceeds that is to _alter data_ in the `"q"` table in an existing record, the `row in "q"` __must exist__; it __won't emit__ an `INSERT` if the _primary key identity_ is __already present__. If the _row does not exist_, for most DBAPI drivers which support reporting the _number of rows affected_ by an `UPDATE`, the _ORM_ will __fail to detect an updated row and raise an error__; otherwise, the data would be _silently ignored_.
>
> A recipe to allow for an `on-the-fly "insert"` of the related row might make use of the `.MapperEvents.before_update` event and look like:
>
> ```
> from sqlalchemy import event
>
>
>@event.listens_for(PtoQ, "before_update")
>def receive_before_update(mapper, connection, target):
>    if target.some_required_attr_on_q is None:
>        connection.execute(q_table.insert(), {"id": target.id})
>```
>
> where above, a row is `INSERT`ed into the *q_table* table by creating an _INSERT_ construct with `Table.insert()`, then executing it using the given _Connection_ which is the __same one being used to emit other SQL for the flush process__. The _user-supplied logic_ would __have to detect__ that the `LEFT OUTER JOIN` _from "p" to "q"_ __does not have an entry__ for the `"q"` side.